In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data/final_dataset/df_final.csv', low_memory=False)

In [6]:
df.shape

(30161, 42)

In [5]:
df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,...,1.44,2.62,1.47,2.85,1.44,2.63,1.47,3.20,1.42,2.78
1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Davydenko N.,...,1.36,3.00,1.29,3.92,1.33,3.00,1.38,3.92,1.33,3.21
2,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Zemlja G.,...,1.67,2.10,1.73,2.22,1.67,2.10,1.73,2.40,1.63,2.21
3,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Troicki V.,...,1.33,3.25,1.38,3.26,1.44,2.63,1.48,3.95,1.40,2.87
4,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Kohlschreiber P.,...,1.50,2.50,1.54,2.63,1.50,2.50,1.57,3.00,1.50,2.52
